# Energy Cost Optimization Tool

The Energy Cost Optimization Tool is designed to analyze and optimize energy costs by combining data from the U.S. Energy Information Administration (EIA) API with insights generated by a Large Language Model (LLM). Users can select specific data routes, set parameters such as frequency, state, sector, and date ranges, and choose relevant data fields for analysis.

This tool provides actionable insights that are aligned with LŌD’s expertise in energy management and cost optimization. By focusing on real-time data and tailored recommendations, the tool reflects LŌD's commitment to helping industries enhance energy efficiency, manage energy resources effectively, and participate in programs like demand response. It supports decision-making in mission-critical environments, helping users improve energy strategies while leveraging LŌD’s broader suite of solutions.

In [ ]:
import sys
import os
import logging
import traceback
from IPython.display import clear_output  # Import to clear output
from dotenv import load_dotenv  # Import to load environment variables from .env

# Set logging level to debug to see more details
logging.basicConfig(level=logging.DEBUG)

# Define a list of necessary packages
required_packages = [
    "ipywidgets",
    "requests",
    "pandas",
    "python-dotenv"  # Used for handling environment variables in the .env file
]

# Install all required dependencies
!pip install -q ipywidgets pandas requests python-dotenv

# Load environment variables from the .env file before changing directories
env_file_path = os.path.abspath('.env')  # Make sure to use absolute path
if os.path.exists(env_file_path):
    load_dotenv(env_file_path)
    logging.info("Loaded environment variables from .env file.")
else:
    logging.warning(f"No .env file found at {env_file_path}")

# Retrieve API keys from environment variables and log them (obscure the key)
EIA_API_KEY = os.getenv("EIA_API_KEY")
CHAT_GPT_API_KEY = os.getenv("CHAT_GPT_API_KEY")
logging.info(f"EIA_API_KEY: {'Loaded' if EIA_API_KEY else 'Not Loaded'}")
logging.info(f"CHAT_GPT_API_KEY: {'Loaded' if CHAT_GPT_API_KEY else 'Not Loaded'}")

if 'google.colab' in sys.modules:
    try:
        # Step 1: Navigate to root to reset the directory
        %cd /content

        # Step 2: Set up the repository path to a temporary directory in /content
        repo_parent_path = '/content/temporary_workspace'
        repo_path = os.path.join(repo_parent_path, 'Energy-Cost-Optimization-Tool')
        
        # Create the temporary directory if it doesn't exist
        if not os.path.exists(repo_parent_path):
            os.makedirs(repo_parent_path)
            logging.info(f"Created new parent directory: {repo_parent_path}")

        # Step 3: Check if the repository already exists and remove it to avoid conflicts
        if os.path.exists(repo_path):
            logging.info("Removing old version of the repository.")
            %rm -rf {repo_path}

        # Step 4: Clone the repository into the new directory
        logging.info("Cloning the repository.")
        !git clone https://github.com/hansnery/Energy-Cost-Optimization-Tool.git {repo_path}
        if not os.path.exists(repo_path):
            raise Exception("Failed to clone the repository.")
        
        %cd {repo_path}
        logging.info("Repository cloned successfully.")

        # Step 5: Install the necessary packages
        logging.info("Installing required dependencies.")
        for package in required_packages:
            %pip install {package}
        logging.info("Dependencies installed successfully.")

        # Step 6: Clear all the previous outputs
        clear_output(wait=True)

        # Step 7: Conditionally load autoreload to avoid redundant loading messages
        try:
            get_ipython().run_line_magic('reload_ext', 'autoreload')
        except:
            get_ipython().run_line_magic('load_ext', 'autoreload')
        get_ipython().run_line_magic('autoreload', '2')

        # Step 8: Import necessary modules from the cloned repository
        logging.info("Importing interface module.")
        from interface import EnergyCostOptimizationInterface
        logging.info("Imported interface module successfully.")

        # Step 9: Initialize and display the interface with API keys if available
        if EIA_API_KEY and CHAT_GPT_API_KEY:
            logging.info("API keys loaded successfully from .env file.")
            interface = EnergyCostOptimizationInterface()
            interface.eia_api_key = EIA_API_KEY
            interface.chat_gpt_api_key = CHAT_GPT_API_KEY
            interface.initialize_apis()
            interface.fetch_routes()
            interface.display_interface()
        else:
            logging.warning("API keys not loaded. Prompting user for input.")
            EnergyCostOptimizationInterface()

    except Exception as e:
        logging.error("An error occurred during setup.")
        traceback.print_exc()
        print(f"Error: {e}")

else:
    print("This script is intended to run in Google Colab.")
    from interface import EnergyCostOptimizationInterface
    if EIA_API_KEY and CHAT_GPT_API_KEY:
        interface = EnergyCostOptimizationInterface()
        interface.eia_api_key = EIA_API_KEY
        interface.chat_gpt_api_key = CHAT_GPT_API_KEY
        interface.initialize_apis()
        interface.fetch_routes()
        interface.display_interface()
    else:
        EnergyCostOptimizationInterface()
